# RFM (pandas)

In [2]:
import pandas as pd
import dask.dataframe as dd
from datetime import datetime
import numpy as np

#import squarify
import matplotlib.pyplot as plt

Leemos el archivo.

In [3]:
df=pd.read_csv('2019-Oct-Nov-transformed.csv')
df

,event_time,event_type,product_id,category_code,brand,price,user_id
0,2019-10-01,purchase,1004856,electronics.smartphone,samsung,130.76,543272936
1,2019-10-01,purchase,1002532,electronics.smartphone,apple,642.69,551377651
2,2019-10-01,purchase,5100816,no category,xiaomi,29.51,514591159
3,2019-10-01,purchase,13800054,furniture.bathroom.toilet,santeri,54.42,555332717
4,2019-10-01,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178
...,...,...,...,...,...,...,...
1659783,2019-11-30,purchase,1004874,electronics.smartphone,samsung,346.70,547804983
1659784,2019-11-30,purchase,1005130,electronics.smartphone,apple,1437.02,515582054
1659785,2019-11-30,purchase,1004767,electronics.smartphone,samsung,235.60,579876821
1659786,2019-11-30,purchase,3701309,appliances.environment.vacuum,polaris,89.32,543733099


In [4]:
df['event_time'] =  pd.to_datetime(df['event_time'], format='%Y-%m-%d')
df

,event_time,event_type,product_id,category_code,brand,price,user_id
0,2019-10-01,purchase,1004856,electronics.smartphone,samsung,130.76,543272936
1,2019-10-01,purchase,1002532,electronics.smartphone,apple,642.69,551377651
2,2019-10-01,purchase,5100816,no category,xiaomi,29.51,514591159
3,2019-10-01,purchase,13800054,furniture.bathroom.toilet,santeri,54.42,555332717
4,2019-10-01,purchase,4804055,electronics.audio.headphone,apple,189.91,524601178
...,...,...,...,...,...,...,...
1659783,2019-11-30,purchase,1004874,electronics.smartphone,samsung,346.70,547804983
1659784,2019-11-30,purchase,1005130,electronics.smartphone,apple,1437.02,515582054
1659785,2019-11-30,purchase,1004767,electronics.smartphone,samsung,235.60,579876821
1659786,2019-11-30,purchase,3701309,appliances.environment.vacuum,polaris,89.32,543733099


### RECENCY

In [5]:
window= np.datetime64('2019-12-01','ns')
window = window.astype("datetime64[D]")

In [6]:
df_recency = df.groupby(by='user_id', as_index=False)['event_time'].max()
df_recency

,user_id,event_time
0,138340325,2019-11-11
1,225644257,2019-11-12
2,253299396,2019-11-06
3,256164170,2019-11-20
4,264649825,2019-10-06
...,...,...
697465,579962677,2019-11-30
697466,579963523,2019-11-30
697467,579964521,2019-11-30
697468,579966375,2019-11-30


In [7]:
df_recency['Recency'] = df_recency['event_time'].apply(lambda x: (window - x).days)
df_recency

,user_id,event_time,Recency
0,138340325,2019-11-11,20
1,225644257,2019-11-12,19
2,253299396,2019-11-06,25
3,256164170,2019-11-20,11
4,264649825,2019-10-06,56
...,...,...,...
697465,579962677,2019-11-30,1
697466,579963523,2019-11-30,1
697467,579964521,2019-11-30,1
697468,579966375,2019-11-30,1


In [8]:
df_recency = df_recency[['user_id','Recency']]
df_recency

,user_id,Recency
0,138340325,20
1,225644257,19
2,253299396,25
3,256164170,11
4,264649825,56
...,...,...
697465,579962677,1
697466,579963523,1
697467,579964521,1
697468,579966375,1


### FREQUENCY

In [9]:
df_frequency = df.drop_duplicates().groupby(by=['user_id'], as_index=False)['event_time'].count()
df_frequency

,user_id,event_time
0,138340325,1
1,225644257,1
2,253299396,1
3,256164170,1
4,264649825,2
...,...,...
697465,579962677,1
697466,579963523,1
697467,579964521,1
697468,579966375,1


In [10]:
df_frequency.rename(columns={'event_time':'Frequency'}, inplace=True)
df_frequency

,user_id,Frequency
0,138340325,1
1,225644257,1
2,253299396,1
3,256164170,1
4,264649825,2
...,...,...
697465,579962677,1
697466,579963523,1
697467,579964521,1
697468,579966375,1


### MONETARY

In [11]:
df_monetary = df.groupby(by='user_id', as_index=False)['price'].sum()
df_monetary

,user_id,price
0,138340325,93.50
1,225644257,40.91
2,253299396,246.85
3,256164170,113.23
4,264649825,1240.04
...,...,...
697465,579962677,1634.38
697466,579963523,457.91
697467,579964521,334.60
697468,579966375,124.11


In [12]:
df_monetary.rename(columns={'price':'Monetary'}, inplace=True)
df_monetary

,user_id,Monetary
0,138340325,93.50
1,225644257,40.91
2,253299396,246.85
3,256164170,113.23
4,264649825,1240.04
...,...,...
697465,579962677,1634.38
697466,579963523,457.91
697467,579964521,334.60
697468,579966375,124.11


### MERGE LOS TRES DF

In [13]:
df_rfm = df_recency.merge(df_frequency, how='inner')
df_rfm

,user_id,Recency,Frequency
0,138340325,20,1
1,225644257,19,1
2,253299396,25,1
3,256164170,11,1
4,264649825,56,2
...,...,...,...
697465,579962677,1,1
697466,579963523,1,1
697467,579964521,1,1
697468,579966375,1,1


In [14]:
df_rfm = df_rfm.merge(df_monetary, how = 'inner')
df_rfm

,user_id,Recency,Frequency,Monetary
0,138340325,20,1,93.50
1,225644257,19,1,40.91
2,253299396,25,1,246.85
3,256164170,11,1,113.23
4,264649825,56,2,1240.04
...,...,...,...,...
697465,579962677,1,1,1634.38
697466,579963523,1,1,457.91
697467,579964521,1,1,334.60
697468,579966375,1,1,124.11


### Eliminamos outliers utilizando el percentil 98.

In [15]:
#Chequeamos máximos y mínimos en Recency (no puede ser menor a 1 ni mayor a 61)
print(df_rfm['Recency'].min())
print(df_rfm['Recency'].max())

1
61


In [16]:
#Chequeamos máximos y mínimos en Frequency
print(df_rfm['Frequency'].min())
print(df_rfm['Frequency'].max())

1
319


In [17]:
#Chequeamos máximos y mínimos en Monetary
print(df_rfm['Monetary'].min())
print(df_rfm['Monetary'].max())

0.77
344153.29


In [18]:
#Para Frequency, calculamos el percentil 98 (q98) y nos quedamos con los valores menores a él.

p98 = df_rfm['Frequency'].quantile(0.98)
print('Percentil 98: ' + str(p98))

df_rfm = df_rfm[df_rfm.Frequency <= p98]
df_rfm

Percentil 98: 10.0


,user_id,Recency,Frequency,Monetary
0,138340325,20,1,93.50
1,225644257,19,1,40.91
2,253299396,25,1,246.85
3,256164170,11,1,113.23
4,264649825,56,2,1240.04
...,...,...,...,...
697465,579962677,1,1,1634.38
697466,579963523,1,1,457.91
697467,579964521,1,1,334.60
697468,579966375,1,1,124.11


In [19]:
#Para Monetary, calculamos el percentil 2 y el percentil 98 (p2 y p98) y nos quedamos con el intervalo entre estos dos valores.
p2 = df_rfm['Monetary'].quantile(0.02)
p98 = df_rfm['Monetary'].quantile(0.98)

print('Percentil 2: ' + str(p2))
print('Percentil 98: ' + str(p98))

df_rfm = df_rfm[df_rfm.Monetary > p2]
df_rfm = df_rfm[df_rfm.Monetary < p98]
df_rfm


Percentil 2: 18.02
Percentil 98: 3538.613599999993


,user_id,Recency,Frequency,Monetary
0,138340325,20,1,93.50
1,225644257,19,1,40.91
2,253299396,25,1,246.85
3,256164170,11,1,113.23
4,264649825,56,2,1240.04
...,...,...,...,...
697465,579962677,1,1,1634.38
697466,579963523,1,1,457.91
697467,579964521,1,1,334.60
697468,579966375,1,1,124.11


In [20]:
df_rfm.reset_index(inplace=True, drop=True)

In [19]:
df_rfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658026 entries, 0 to 658025
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    658026 non-null  int64  
 1   Recency    658026 non-null  int64  
 2   Frequency  658026 non-null  int64  
 3   Monetary   658026 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 20.1 MB


### Asignamos quintiles

In [21]:
rfm = df_rfm

rfm['R'], bins = pd.qcut(rfm['Recency'], q=5, labels=[5,4,3,2,1], retbins=True)

In [22]:
condition = [(rfm.Frequency == 1) | (rfm.Frequency == 2),
            (rfm.Frequency == 3) | (rfm.Frequency == 4),
            (rfm.Frequency == 5) | (rfm.Frequency == 6),
            (rfm.Frequency == 7) | (rfm.Frequency == 8)]
option = [1,2,3,4]
rfm['F'] = np.select(condition, option, default= 5)

rfm['M'] = pd.qcut(rfm['Monetary'], q=5, labels=[1,2,3,4,5]) 
rfm

,user_id,Recency,Frequency,Monetary,R,F,M
0,138340325,20,1,93.50,3,1,1
1,225644257,19,1,40.91,3,1,1
2,253299396,25,1,246.85,3,1,3
3,256164170,11,1,113.23,5,1,2
4,264649825,56,2,1240.04,1,1,5
...,...,...,...,...,...,...,...
658021,579962677,1,1,1634.38,5,1,5
658022,579963523,1,1,457.91,5,1,4
658023,579964521,1,1,334.60,5,1,3
658024,579966375,1,1,124.11,5,1,2


In [23]:
rfm[rfm['user_id']==546982605]

,user_id,Recency,Frequency,Monetary,R,F,M
378287,546982605,11,3,860.41,5,2,5


In [24]:
rfm["RFM_SCORE"] = (rfm['R'].astype(str) + 
                    rfm['F'].astype(str) +
                    rfm['M'].astype(str))
rfm

,user_id,Recency,Frequency,Monetary,R,F,M,RFM_SCORE
0,138340325,20,1,93.50,3,1,1,311
1,225644257,19,1,40.91,3,1,1,311
2,253299396,25,1,246.85,3,1,3,313
3,256164170,11,1,113.23,5,1,2,512
4,264649825,56,2,1240.04,1,1,5,115
...,...,...,...,...,...,...,...,...
658021,579962677,1,1,1634.38,5,1,5,515
658022,579963523,1,1,457.91,5,1,4,514
658023,579964521,1,1,334.60,5,1,3,513
658024,579966375,1,1,124.11,5,1,2,512


### A partir del RFM_SCORE, mapeamos los segmentos.

In [25]:
seg_map = pd.read_csv('segmentos_rfm.csv', usecols=['RFM_SCORE','segment'])
seg_map['RFM_SCORE']= seg_map.RFM_SCORE.astype(str)
seg_map.head()

,RFM_SCORE,segment
0,111,Lost
1,112,Lost
2,113,Lost
3,114,At Risk
4,115,At Risk


In [26]:
rfm = rfm.merge(seg_map, on='RFM_SCORE')
rfm 

,user_id,Recency,Frequency,Monetary,R,F,M,RFM_SCORE,segment
0,138340325,20,1,93.50,3,1,1,311,About To Sleep
1,225644257,19,1,40.91,3,1,1,311,About To Sleep
2,282274853,23,1,79.15,3,1,1,311,About To Sleep
3,310402868,20,1,77.22,3,1,1,311,About To Sleep
4,314102803,19,1,61.01,3,1,1,311,About To Sleep
...,...,...,...,...,...,...,...,...,...
658021,554615276,55,7,138.70,1,4,2,142,At Risk
658022,518195785,47,10,156.99,1,5,2,152,At Risk
658023,526169362,58,10,261.25,1,5,3,153,At Risk
658024,539559331,45,9,305.41,1,5,3,153,At Risk


### Exportamos el dataset

In [27]:
rfm.to_csv('RFM Resultados.csv')

### How to read this table?
Champions
There are 90063 people in this group.
They do not shop for an average of 4.93 days.
They shopped an average of 4.67 times.
They earned an average of 1753.93 units of money.

In [102]:
rfm[["segment", "Recency", "Frequency", "Monetary"]].groupby("segment").agg(["mean", "count","max"])

Recency             Frequency                 Monetary  \
                         mean   count max      mean   count max         mean   
segment                                                                        
About To Sleep      31.459617  126773  61  1.160216  126773   7   125.764235   
At Risk             51.142968   47479  61  2.015038   47479  10   908.988345   
Can't Lose Them     36.756854   46285  61  1.966901   46285  10   907.469331   
Champions            5.272586   30992  15  4.802013   30992  10  1323.636755   
Hibernating         28.586820    6525  44  3.401533    6525   6   219.284564   
Lost                52.496197   85845  61  1.117141   85845   4   144.675115   
Loyal Customers     17.190488    8347  28  5.988619    8347  10  1715.353436   
Need Attention      22.023579   31257  43  1.779057   31257  10   806.904988   
New Customers        5.612618   76604  11  1.332907   76604   2   432.109916   
Potential Loyalist  14.163667   78006  28  2.223624   78006  10   500.427835   
Promising           14.046742  119913  28  1.311651  119913  10   417.853932   

                                     
                     count      max  
segment                              
About To Sleep      126773   347.45  
At Risk              47479  3534.87  
Can't Lose Them      46285  3537.69  
Champions            30992  3537.73  
Hibernating           6525   347.45  
Lost                 85845   347.45  
Loyal Customers       8347  3538.60  
Need Attention       31257  3537.34  
New Customers        76604  3538.08  
Potential Loyalist   78006  3538.51  
Promising           119913  3533.38